## Calculating correlations of variables to Z500 and T850

In [1]:
import xarray as xr

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
DATADIR = '/rds/general/user/mc4117/home/WeatherBench/data/'  

In [3]:
# list of all variables in dataset
! ls $DATADIR

10m_u_component_of_wind  potential_vorticity	       total_cloud_cover
10m_v_component_of_wind  relative_humidity	       total_precipitation
2m_temperature		 specific_humidity	       u_component_of_wind
constants		 temperature		       v_component_of_wind
geopotential		 temperature_850	       vorticity
geopotential_500	 toa_incident_solar_radiation


In [18]:
# functions to calculate the covariance between variables
def covariance(x, y, dim=None):
    x_arr = x
    y_arr = y
    valid_values = x_arr.notnull() & y_arr.notnull()
    valid_count = valid_values.sum(dim)

    demeaned_x = (x_arr - x_arr.mean(dim)).fillna(0)
    demeaned_y = (y_arr - y_arr.mean(dim)).fillna(0)
    
    return xr.dot(demeaned_x, demeaned_y, dims=dim) / valid_count

def correlation(x, y, dim=None):
    # dim should default to the intersection of x.dims and y.dims
    return covariance(x, y, dim) / (x.std(dim) * y.std(dim))

In [5]:
def color_negative_red(value):
    # color table values depending on value
    if abs(value) < 0.25:
        color = 'red'
    elif abs(value) > 0.5:
        color = 'green'
    else:
        color = 'black'

    return 'color: %s' % color

In [6]:
# read in all datasets
u_wind_10m = xr.open_mfdataset(f'{DATADIR}10m_u_component_of_wind/*.nc', combine='by_coords')
v_wind_10m = xr.open_mfdataset(f'{DATADIR}10m_v_component_of_wind/*.nc', combine='by_coords')
u_wind = xr.open_mfdataset(f'{DATADIR}u_component_of_wind/*.nc', combine='by_coords')
v_wind = xr.open_mfdataset(f'{DATADIR}v_component_of_wind/*.nc', combine='by_coords')
temp_2m = xr.open_mfdataset(f'{DATADIR}2m_temperature/*.nc', combine='by_coords')
constants = xr.open_mfdataset(f'{DATADIR}constants/*.nc', combine='by_coords')
geo_500 = xr.open_mfdataset(f'{DATADIR}geopotential_500/*.nc', combine='by_coords')
pot_vort = xr.open_mfdataset(f'{DATADIR}potential_vorticity/*.nc', combine='by_coords')
rel_hum = xr.open_mfdataset(f'{DATADIR}relative_humidity/*.nc', combine='by_coords')
spec_hum = xr.open_mfdataset(f'{DATADIR}specific_humidity/*.nc', combine='by_coords')
temp850 = xr.open_mfdataset(f'{DATADIR}temperature_850/*.nc', combine='by_coords')
solar = xr.open_mfdataset(f'{DATADIR}toa_incident_solar_radiation/*.nc', combine='by_coords')
cloud_cover = xr.open_mfdataset(f'{DATADIR}total_cloud_cover/*.nc', combine='by_coords')
precip = xr.open_mfdataset(f'{DATADIR}total_precipitation/*.nc', combine='by_coords')
vort = xr.open_mfdataset(f'{DATADIR}vorticity/*.nc', combine='by_coords')

In [16]:
# Select variables that want to calculate correlation with.
# If the variable is multileveled we select 500hPa and 850hPa as these are the levels trying to predict.
# A more thorough analysis of the correlations at all levels is conducted in "correlation_levels.ipynb" in the same folder
variables = [
             u_wind.u.sel(level=500), u_wind.u.sel(level=850), v_wind.v.sel(level=500), v_wind.v.sel(level=850), 
             u_wind_10m.u10, v_wind_10m.v10, temp_2m.t2m, geo_500.z, pot_vort.pv.sel(level = 500), pot_vort.pv.sel(level=850),
             rel_hum.r.sel(level = 500), rel_hum.r.sel(level=850), spec_hum.q.sel(level=500),
             spec_hum.q.sel(level=850), temp850.t, solar.tisr, cloud_cover.tcc, precip.tp, 
             vort.vo.sel(level=500), vort.vo.sel(level=850)
            ]
variables_str = [
             'u_wind_500', 'v_wind_500', 'u_wind_850', 'v_wind_850', 'u_wind_10m', 'v_wind_10m', 
             'temp_2m', 'geo_500', 'pot_vort_500', 'pot_vort_850',
             'rel_hum_500', 'rel_hum_850', 'spec_hum_500',
             'spec_hum_850', 'temp850', 'solar', 'cloud_cover', 'precip', 
             'vort_500', 'vort_850'
            ]

# function to construct correlation tables
def correlation_variables_table(data_list, y, lead_time, north = False, south = False):
    correlation_list = []
    
    y_corr = y.sel(time = slice(y.time[lead_time], None))
    
    for i in variables:
        if north:
            i_known = i.sel(lat = slice(0, 90), time = slice(None, i.time[-(lead_time +1)])) 
        elif south:
            i_known = i.sel(lat = slice(-90, 0), time = slice(None, i.time[-(lead_time +1)])) 
        else:
            i_known = i.sel(time = slice(None, i.time[-(lead_time +1)])) 
        correlation_list.append(correlation(i_known, y_corr).values)    

    return correlation_list


In [19]:
# construct correlation table with variables selected T850 3 days ahead
temp850_corr = correlation_variables_table(variables, temp850.t, lead_time = 72)
# construct correlation table with variables selected Z500 3 days ahead
geo500_corr = correlation_variables_table(variables, geo_500.z, lead_time = 72)


temp done


In [20]:
# visualise correlation table for T850 3days
df = pd.concat([pd.DataFrame(variables_str, columns = ['Variable names']), pd.DataFrame(temp850_corr, columns = ['Corr_tmp_850 full'])], axis = 1)

df.style.applymap(color_negative_red, subset=['Corr_tmp_850 full'])

,Variable names,Corr_tmp_850 full
0,u_wind_500,-0.059584
1,v_wind_500,-0.100996
2,u_wind_850,-0.015567
3,v_wind_850,-0.042132
4,u_wind_10m,-0.135853
5,v_wind_10m,-0.050571
6,temp_2m,0.946909
7,geo_500,0.934707
8,pot_vort_500,0.150038
9,pot_vort_850,0.231432


In [21]:
# visualise correlation table for Z500 3days
df = pd.concat([pd.DataFrame(variables_str, columns = ['Variable names']), pd.DataFrame(geo500_corr, columns = ['Corr_geo_500 full'])], axis = 1)

df.style.applymap(color_negative_red, subset=['Corr_geo_500 full'])

,Variable names,Corr_geo_500 full
0,u_wind_500,-0.067465
1,v_wind_500,-0.147559
2,u_wind_850,-0.001903
3,v_wind_850,-0.029064
4,u_wind_10m,-0.184053
5,v_wind_10m,-0.032521
6,temp_2m,0.881828
7,geo_500,1.000012
8,pot_vort_500,0.202548
9,pot_vort_850,0.219346


In [22]:
# construct correlation table with variables selected T850 5 days ahead
temp850_corr = correlation_variables_table(variables, temp850.t, lead_time = 120)
# construct correlation table with variables selected Z500 5 days ahead
geo500_corr = correlation_variables_table(variables, geo_500.z, lead_time = 120)

temp done


In [23]:
# visualise correlation table for T850 3days
df = pd.concat([pd.DataFrame(variables_str, columns = ['Variable names']), pd.DataFrame(temp850_corr, columns = ['Corr_tmp_850 full'])], axis = 1)

df.style.applymap(color_negative_red, subset=['Corr_tmp_850 full'])

,Variable names,Corr_tmp_850 full
0,u_wind_500,-0.059588
1,v_wind_500,-0.100973
2,u_wind_850,-0.015579
3,v_wind_850,-0.042146
4,u_wind_10m,-0.135824
5,v_wind_10m,-0.050574
6,temp_2m,0.946927
7,geo_500,0.934722
8,pot_vort_500,0.150140
9,pot_vort_850,0.231494


In [24]:
# visualise correlation table for Z500 3days
df = pd.concat([pd.DataFrame(variables_str, columns = ['Variable names']), pd.DataFrame(geo500_corr, columns = ['Corr_geo_500 full'])], axis = 1)

df.style.applymap(color_negative_red, subset=['Corr_geo_500 full'])

,Variable names,Corr_geo_500 full
0,u_wind_500,-0.067485
1,v_wind_500,-0.147548
2,u_wind_850,-0.001903
3,v_wind_850,-0.029069
4,u_wind_10m,-0.184033
5,v_wind_10m,-0.032517
6,temp_2m,0.881841
7,geo_500,1.000023
8,pot_vort_500,0.202622
9,pot_vort_850,0.219394


In [ ]:
# Constants are constant in time and therefore it does not make sense to look at different lead times
# Instead we look at the north-south split

geo_500_north = geo_500.sel(lat=slice(0, 90))
geo_500_south = geo_500.sel(lat=slice(-90, 0))

t850_north = temp850.sel(lat=slice(0, 90))
t850_south = temp850.sel(lat=slice(-90, 0))

In [12]:
# correlation with Z500 and constants
constant_variables_str = []
corr_z500_const = []
corr_z500_north_const = []
corr_z500_south_const = []

constant_variables_str.append('orography')
corr_z500_north_const.append(np.mean([correlation(constants.orography.sel(lat = slice(0,90)), geo_500_north.z.isel(time = np.int(100*i))).values for i in range(np.int(len(geo_500_north.z)/100))]))
corr_z500_const.append(np.mean([correlation(constants.orography, geo_500.z.isel(time = np.int(100*i))).values for i in range(np.int(len(geo_500.z)/100))]))
corr_z500_south_const.append(np.mean([correlation(constants.orography.sel(lat = slice(-90, 0)), geo_500_south.z.isel(time = np.int(100*i))).values for i in range(np.int(len(geo_500_south.z)/100))]))


constant_variables_str.append('lsm')
corr_z500_north_const.append(np.mean([correlation(constants.lsm.sel(lat = slice(0,90)), geo_500_north.z.isel(time = np.int(100*i))).values for i in range(np.int(len(geo_500_north.z)/100))]))
corr_z500_const.append(np.mean([correlation(constants.lsm, geo_500.z.isel(time = np.int(100*i))).values for i in range(np.int(len(geo_500.z)/100))]))
corr_z500_south_const.append(np.mean([correlation(constants.lsm.sel(lat = slice(-90, 0)), geo_500_south.z.isel(time = np.int(100*i))).values for i in range(np.int(len(geo_500_south.z)/100))]))


constant_variables_str.append('slt')
corr_z500_north_const.append(np.mean([correlation(constants.slt.sel(lat = slice(0,90)), geo_500_north.z.isel(time = np.int(100*i))).values for i in range(np.int(len(geo_500_north.z)/100))]))
corr_z500_const.append(np.mean([correlation(constants.slt, geo_500.z.isel(time = np.int(100*i))).values for i in range(np.int(len(geo_500.z)/100))]))
corr_z500_south_const.append(np.mean([correlation(constants.slt.sel(lat = slice(-90, 0)), geo_500_south.z.isel(time = np.int(100*i))).values for i in range(np.int(len(geo_500_south.z)/100))]))


constant_variables_str.append('lat')
corr_z500_north_const.append(np.mean([correlation(constants.lat2d.sel(lat = slice(0,90)), geo_500_north.z.isel(time = np.int(100*i))).values for i in range(np.int(len(geo_500_north.z)/100))]))
corr_z500_const.append(np.mean([correlation(constants.lat2d, geo_500.z.isel(time = np.int(100*i))).values for i in range(np.int(len(geo_500.z)/100))]))
corr_z500_south_const.append(np.mean([correlation(constants.lat2d.sel(lat = slice(-90,0)), geo_500_south.z.isel(time = np.int(100*i))).values for i in range(np.int(len(geo_500_south.z)/100))]))

constant_variables_str.append('lon')
corr_z500_north_const.append(np.mean([correlation(constants.lon2d.sel(lat = slice(0,90)), geo_500_north.z.isel(time = np.int(100*i))).values for i in range(np.int(len(geo_500_north.z)/100))]))
corr_z500_const.append(np.mean([correlation(constants.lon2d, geo_500.z.isel(time = np.int(100*i))).values for i in range(np.int(len(geo_500.z)/100))]))
corr_z500_south_const.append(np.mean([correlation(constants.lon2d.sel(lat = slice(-90, 0)), geo_500_south.z.isel(time = np.int(100*i))).values for i in range(np.int(len(geo_500_south.z)/100))]))

df = pd.concat([pd.DataFrame(constant_variables_str, columns = ['Variable names']),pd.DataFrame(corr_z500_const, columns = ['Corr z500'])], axis = 1)

df.style.applymap(color_negative_red, subset=['Corr z500'])


,Variable names,Corr z500
0,orography,-0.325298
1,lsm,-0.189865
2,slt,0.047839
3,lat,0.236026
4,lon,0.006956


In [13]:
# visualise table with correlation between Z500 and constants
df = pd.concat([pd.DataFrame(constant_variables_str, columns = ['Variable names']),pd.DataFrame(corr_z500_const, columns = ['Corr z500']), pd.DataFrame(corr_z500_north_const, columns = ['Corr z500 north']), pd.DataFrame(corr_z500_south_const, columns = ['Corr z500 south'])], axis = 1)

df.style.applymap(color_negative_red, subset=['Corr z500', 'Corr z500 north', 'Corr z500 south'])

,Variable names,Corr z500,Corr z500 north,Corr z500 south
0,orography,-0.325298,0.049917,-0.475823
1,lsm,-0.189865,-0.010477,-0.364817
2,slt,0.047839,0.045921,-0.014253
3,lat,0.236026,-0.895613,0.928474
4,lon,0.006956,-0.007477,0.017626


In [14]:
# correlation with T850 and constants

constant_variables_str = []
corr_t850_const = []
corr_t850_north_const = []
corr_t850_south_const = []

constant_variables_str.append('orography')
corr_t850_north_const.append(np.mean([correlation(constants.orography.sel(lat = slice(0,90)), t850_north.t.isel(time = np.int(100*i))).values for i in range(np.int(len(t850_north.t)/100))]))
corr_t850_const.append(np.mean([correlation(constants.orography, temp850.t.isel(time = np.int(100*i))).values for i in range(np.int(len(temp850.t)/100))]))
corr_t850_south_const.append(np.mean([correlation(constants.orography.sel(lat = slice(-90,0)), t850_south.t.isel(time = np.int(100*i))).values for i in range(np.int(len(t850_south.t)/100))]))


constant_variables_str.append('lsm')
corr_t850_north_const.append(np.mean([correlation(constants.lsm.sel(lat = slice(0,90)), t850_north.t.isel(time = np.int(100*i))).values for i in range(np.int(len(t850_north.t)/100))]))
corr_t850_const.append(np.mean([correlation(constants.lsm, temp850.t.isel(time = np.int(100*i))).values for i in range(np.int(len(temp850.t)/100))]))
corr_t850_south_const.append(np.mean([correlation(constants.lsm.sel(lat = slice(-90,0)), t850_south.t.isel(time = np.int(100*i))).values for i in range(np.int(len(t850_south.t)/100))]))


constant_variables_str.append('slt')
corr_t850_north_const.append(np.mean([correlation(constants.slt.sel(lat = slice(0,90)), t850_north.t.isel(time = np.int(100*i))).values for i in range(np.int(len(t850_north.t)/100))]))
corr_t850_const.append(np.mean([correlation(constants.slt, temp850.t.isel(time = np.int(100*i))).values for i in range(np.int(len(temp850.t)/100))]))
corr_t850_south_const.append(np.mean([correlation(constants.slt.sel(lat = slice(-90,0)), t850_south.t.isel(time = np.int(100*i))).values for i in range(np.int(len(t850_south.t)/100))]))

constant_variables_str.append('lat')
corr_t850_north_const.append(np.mean([correlation(constants.lat2d.sel(lat = slice(0,90)), t850_north.t.isel(time = np.int(100*i))).values for i in range(np.int(len(t850_north.t)/100))]))
corr_t850_const.append(np.mean([correlation(constants.lat2d, temp850.t.isel(time = np.int(100*i))).values for i in range(np.int(len(temp850.t)/100))]))
corr_t850_south_const.append(np.mean([correlation(constants.lat2d.sel(lat = slice(-90,0)), t850_south.t.isel(time = np.int(100*i))).values for i in range(np.int(len(t850_south.t)/100))]))

constant_variables_str.append('lon')
corr_t850_north_const.append(np.mean([correlation(constants.lon2d.sel(lat = slice(0,90)), t850_north.t.isel(time = np.int(100*i))).values for i in range(np.int(len(t850_north.t)/100))]))
corr_t850_const.append(np.mean([correlation(constants.lon2d, temp850.t.isel(time = np.int(100*i))).values for i in range(np.int(len(temp850.t)/100))]))
corr_t850_south_const.append(np.mean([correlation(constants.lon2d.sel(lat = slice(-90,0)), t850_south.t.isel(time = np.int(100*i))).values for i in range(np.int(len(t850_south.t)/100))]))

# visualise table
df2 = pd.concat([pd.DataFrame(constant_variables_str, columns = ['Variable names']),pd.DataFrame(corr_t850_const, columns = ['Corr t850']), pd.DataFrame(corr_t850_north_const, columns = ['Corr t850 north']), pd.DataFrame(corr_t850_south_const, columns = ['Corr t850 south'])], axis = 1)

df2.style.applymap(color_negative_red, subset=['Corr t850', 'Corr t850 north', 'Corr t850 south'])


,Variable names,Corr t850,Corr t850 north,Corr t850 south
0,orography,-0.385886,0.084928,-0.627422
1,lsm,-0.214283,0.037092,-0.451531
2,slt,0.040490,0.085191,-0.046446
3,lat,0.167379,-0.895283,0.940024
4,lon,-0.001363,-0.064480,0.041545
